# INTRODUCTION
Jupyter notebook to query the harvested metadata records from the IISG bibliographic materials (biblio)

This notebook makes it possible to get overviews and query the metadata records of the International Institute of Social History (IISG) Bibliographic materials ("Biblio"). It uses as source the file "converted.csv" obtained via metadata harvesting using the scripts in this repository (https://github.com/lilimelgar/iisg-metadata-overviews).  It contains MARC records from the OAIPMH endpoint. 
The file contains one record per row, and each marc property (field and subfield) is in a column.

Note: the data includes only metadata records at the "item" level.

Created by Liliana Melgar (April, 2024).

# A. Set up

## Import the required python libraries 
*(nothing to change)*

In [ ]:
import pandas as pd
import numpy as np
import csv
import re

from IPython.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# to add timestamp to file names
import time
# import os.path to add paths to files
import os

# disable warning for false positive on chained assignment
pd.options.mode.chained_assignment = None  # default='warn'

## Set the path to the csv file 
*nothing to change if you cloned the repository. If you downloaded the file only ("biblio_as_csv.gzip"), then set here the path to where you have downloaded the file*

In [ ]:
# path to where the relevant data is located
# biblio
script_dir = os.getcwd()  # Gets the current working directory
project_root = os.path.abspath(os.path.join(script_dir, "..", ".."))  # Moves up two levels to reach 'repo'
data_directory_biblio = os.path.join(project_root, "data", "biblio")
data_converted_biblio = os.path.join(data_directory_biblio, 'converted')
data_downloads_biblio = os.path.join(data_directory_biblio, 'downloads') #path to the folder where the reports will be downloaded

## Read the csv file as a pandas dataframe
*nothing to change here, just be patient, IT TAKES LONG TO LOAD (around started at 19.00h and finished sometime before 20:48h same day)*

In [ ]:
# read csv as dataframe
biblio_df_v0 = pd.read_csv(f'{data_converted_biblio}/biblio_as_csv_per_field.gzip', sep="\t", compression='gzip', low_memory=False)
# low_memory=False was set after this warning message: "/var/folders/3y/xbjxw0b94jxg6x2bcbyjsmmcgvnf7q/T/ipykernel_987/2912965462.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False."

In [ ]:
# biblio_df_v0.describe()

## Inspect if import was correct
First overview: all fields and data types
Execute the cell and view the general information of the data, which includes the Columns (marc properties with subfields), the Non-Null Count (i.e., how many cells have values; for example: if a cell says "1 non-null" it means that only one row has a value); and the Data type (object (i.e., a string or a combination of data types), a float or an integer).
- Keep in mind that the MARC labels have 3 characters, and that the fourth character can be an indicator or a subfield. For example: 1000 is Marc label 100 with indicator 0. And 100a is Marc label 100 with subfield a.

In [ ]:
biblio_df_v0.info(verbose = True, show_counts = True)

## Columns (documentation)
Ideally, each field above would have a definition explaining what it means and what kind of values does it contain (in relation to the conventions for creating IISG metadata). That documentation can exist somewhere else (e.g., on Confluence), but this could be a place to start updating or writing those definitions since here one can see the data that they contain in detail.
For now, we can consult the MARC21 documentation which explains what each field label means for Bibliographic data: https://www.loc.gov/marc/bibliographic/

## Prepare the data for search
Because we know that the data doesn't have proper numerical values to be computed, we rather convert all values to strings in order to facilitate querying. This also includes filling in empty values with a standard string: "null"
*(nothing to change here)*

In [ ]:
# convert datatypes and fill in empty values
df_columns = biblio_df_v0.columns
for column in df_columns:
    dataType = biblio_df_v0.dtypes[column]
    if dataType == np.float64:
        biblio_df_v0[column] = biblio_df_v0[column].fillna('null')
        biblio_df_v0[column] = biblio_df_v0[column].astype(str)
    if dataType == np.int_:
        biblio_df_v0[column] = biblio_df_v0[column].fillna('null')
        biblio_df_v0[column] = biblio_df_v0[column].astype(str)
    if dataType == object:
        biblio_df_v0[column] = biblio_df_v0[column].fillna('null')
        biblio_df_v0[column] = biblio_df_v0[column].astype(str)

In [ ]:
# create a copy
biblio_df = biblio_df_v0.copy()

In [ ]:
# # save the csv
# biblio_df.to_csv('biblio_all.csv.gz', index=False, compression='gzip')

In [ ]:
# Check again the general information of the data after having filled in the emtpy values and converted the data types
biblio_df.info(verbose = True, show_counts = True)

# Get a glimpse of the data

## First rows
Here you can see a sample of the records, one per line. You can change the value "10" to any other desired size for your sample, preferably not too big. You can also use "tail" instead of "head" to see the records in the last rows.
- Keep in mind to scroll horizontally and vertically to see the entire record.
- NaN means that the cell is empty.
- Arbitrarily, some cells above, we decided that the omega "Ω" would be the separator for multi-value cells.

In [ ]:
biblio_df.head(5)

## Size (shape) of the data
Here you can see how many rows (first value) and how many columns (second value) are in the data.

In [ ]:
# an ok shape would be around 1207000, 238)
biblio_df.shape

## Unique values
Here you can see a general description of the data, including how many unique values are per column.

In [ ]:
# describe the dataframe
biblio_df.describe()

In [ ]:
# download the file
# name_file = 'biblio_all_harvested20241117_compr'
# biblio_df.to_csv(f'{data_downloads}/{name_file}.csv', index=False, compression='gzip') # if too big, use compression='gzip'

## Check the values in one column (marc property)
At this point you may be curious to know which values are in one column. It may be interesting to observe those that have very few unique values, because they can contain wrong data or wrong columns.
- You can change the field inside the quotation marcs for any other field of interest.

In [ ]:
biblio_df['02e'].unique().tolist()

If you want to see which record contains those values or wrong columns, you can use:

In [ ]:
# check records with non-empty values in a specific column

check_problem1 = biblio_df[~biblio_df['02e'].str.contains("null", case=False, regex=True)]
check_problem1.head(10)

## Inspect one record
If you are interested to see only one record, you can do so by using the recordId (TCN in 001)

In [ ]:
# # TEST (see one record)
# # check if a string value exists in a column (the string is exactly the same)
# # test_exact = biblio_df[biblio_df['651a'] == '1362253']
# # test_exact = biblio_df[biblio_df['651a'] == 'Srebrenica (Yugoslavia)']

check_tcn = '1488082' #'1466360'
check_record = biblio_df[biblio_df['001'] == check_tcn]
check_record

In [ ]:
check_record.shape

In [ ]:
# # You may want to dowload the table above to an excel file for further inspection:

# # choose any name for your file, the file will go to the ../data/downloads folder.
# name_file = 'biblio_651a_Srebrenica'

# test_exact.to_excel(f'{data_downloads}/{name_file}.xlsx')

# Create a slice/subset with certain column(s)/field(s)
At this point you may have thought that you could perhaps correct some of the records which contain an inconsistent value. For example, in the first version of this data, if you queried above for "biblio_df['100e'].unique()" you may have obtained certain values. You may decide that you want to change one or some of them into another value. But for this, you need the TCN (record Id) numbers. The command below facilitates creating a subset with the TCN and the field of interest.

## Test first

In [ ]:
# slice for classification fields (690)
test1 = biblio_df[['001','690']]
test1.info(verbose=True)

In [ ]:
# see the non-null fields
test2 = test1[~test1['690'].str.contains("null", case=False, regex=True)]
test2.head(10)

In [ ]:
test2.shape

## Divide the main df into two dfs (emtpy/non-empty)

In [ ]:
# # create subset with record Id and record of interest, here enter the name of the field(s) that you are interested in separated by commas, each field has to be within single quotation marks, e.g., biblio_df[['001','100e', '110e']]
# # field_subset_df = biblio_df[['001','090a','901a','245a','245b','260a','852p','852j','866a','902a','leader']] #--> For LA periodicals
# # field_subset_df = biblio_df[['001','245a','245b','6510','651a','695g','leader']] #--> For geographic terms exploration
# # field_subset_df = biblio_df[['001','090a','901a','245a','245b','260a','852a','852b','852c','852j','852n','852p','866a','902a','leader']] #--> For periodicals Simon report

## CREATE SUBSET FOR SUBJECT TERMS ANALYSIS (columns starting with 6)
# df_columns = biblio_df.columns
# columns_subjects = []
# for column in biblio_df.columns:
#     if column.startswith("6"):
#         columns_subjects.append(column)
# field_subset_df = biblio_df[['001', 'leader'] + columns_subjects] #--> For av thesaurus terms

# CREATE A SUBSET WITH "690" ONLY to evaluate subject terms (classification)
field_subset_df_v0 = biblio_df[['001', 'leader','690']] #--> For classification terms
# one dataframe for rows with values
field_subset_df_v1 = field_subset_df_v0[field_subset_df_v0['690'].str.lower() != 'null'] #fields with values
# one dataframe for empty rows
field_subset_df_vb = field_subset_df_v0[field_subset_df_v0['690'].str.lower() == 'null'] #fields with empty values (won't be used in this case)

# # CREATE A SUBSET WITH "100" ONLY to evaluate persons
# field_subset_df_v0 = biblio_df[['001', 'leader','100']] #--> For persons
# # one dataframe for rows with values
# field_subset_df_v1 = field_subset_df_v0[field_subset_df_v0['100'].str.lower() != 'null'] #to exclude empty values
# # one dataframe for empty rows
# field_subset_df_vb = field_subset_df_v0[field_subset_df_v0['100'].str.lower() == 'null'] #to exclude empty values (not used in this case)

# field_subset_df.head(10)

In [ ]:
# # field_subset_df.info(verbose=True)
# field_subset_df.describe()
# field_subset_df_vb.head()

In [ ]:
field_subset_df_v2 = field_subset_df_v1.reset_index(drop=True)

In [ ]:
field_subset_df_v2.head(10)

In [ ]:
field_subset_df_v2.tail(10)

In [ ]:
field_subset_df_v2.info(verbose=True)

In [ ]:
# check again the number of unique values in your subset
field_subset_df_v2.describe()

## Prepare non-empty subset 
to have one value per row and one subfield per column

### Check separator

In [ ]:
# # Get the longest cell (to get the most problematic as example)
# # Convert all cells to string and get their lengths
# lengths = field_subset_df_v2.astype(str).map(len)

# # Find position (row, col) of the max length
# max_row, max_col = lengths.stack().idxmax()

# # Get the value from the original DataFrame
# longest_cell = field_subset_df_v2.loc[max_row, max_col]

# print(f"Longest cell is in row {max_row}, column '{max_col}' with length {len(str(longest_cell))}")
# print("Value:", longest_cell)

In [ ]:

# Get the cell where the separator occurs most frequently (to get the most problematic as example)
char = '¶'

# Count how many times 'e' appears in each cell (as string)
char_counts = field_subset_df_v2.astype(str).map(lambda x: x.count(char))

# Find the position of the max count
max_row, max_col = char_counts.stack().idxmax()

# Get the value from the original DataFrame
cell_value = field_subset_df_v2.loc[max_row, max_col]
count = char_counts.loc[max_row, max_col]

print(f"The character '{char}' appears most in row {max_row}, column '{max_col}' ({count} times)")
print("Cell content:", cell_value)

In [ ]:
# get that record Id using row index number
# field_subset_df_v2.iloc[182696]
field_subset_df_v2.iloc[18941]

In [ ]:
# Check the record with the longest value using the TCN (001)
# query_value_aprox = field_subset_df_v2[field_subset_df_v2['001'] == '1080191'].copy()
# query_value_aprox
# test_exact4 = field_subset_df_v2[field_subset_df_v2['001'] == '1466360'] 

test_exact4 = field_subset_df_v2[field_subset_df_v2['001'] == '130229']
test_exact4

### Split multi-valued cells

In [ ]:
# Split into different rows (e.g., explode)

# Step 1: Split the column using "Ω" as the separator
# field_subset_df_v2["100"] = field_subset_df_v2["100"].str.split("¶")
field_subset_df_v2["690"] = field_subset_df_v2["690"].str.split("¶")

# Step 2: Explode the list into multiple rows
# field_subset_df_v2 = field_subset_df_v2.explode("100", ignore_index=True)
field_subset_df_v2 = field_subset_df_v2.explode("690", ignore_index=True)

field_subset_df_v2.head(10)

In [ ]:
# To test if the record is split correctly
test_exact5 = field_subset_df_v2[field_subset_df_v2['001'] == '130229']
test_exact5

In [ ]:
# check the shape, this should be bigger than the number of records
field_subset_df_v2.shape

In [ ]:
field_subset_df_v3 = field_subset_df_v2.reset_index(drop=True)

### Split columns per subfield
This will make that every subfield goes to its own column, the separator between the subfield label and the value should be checked carefully, I used before ":" but this caused problems since some values have ":" in them, thus, now I use also the quotation mark '":'

In [ ]:
# THESE LINES WILL CONVERT FROM THIS FORMAT IN "DATA" to a format in which each subfield is in a separate column 
# with the column name equal to the subfield name, filling in with null the cells where the subfield doesn't exist
# data = {
#     "id": ["001", "002", "003", "004"],
#     "values": ['"a":lore;"b":ipsum', 
#                '"x":nomine;"a":ipsum', 
#                '"x":example;"c":dei',
#                '"b":test;"y":test2']
# }
##########################

# # FOR PERSONS (100)
# # Parse values into a dictionary-like structure
# field_subset_df_v2["parsed"] = field_subset_df_v2["100"].apply(lambda x: {kv.split(":")[0]: kv.split(":")[1] for kv in x.split("⑄")} if isinstance(x, str) else {})
# # Extract all unique keys (column names)
# all_keys = sorted(set(k for d in field_subset_df_v2["parsed"] for k in d.keys()))
# # all_keys
# # Create new columns with values only (remove key names)
# for key in all_keys:
#     field_subset_df_v2[key] = field_subset_df_v2["parsed"].apply(lambda d: d[key] if key in d else "null")
# # df
# # Keep only relevant columns
# field_subset_df_v3 = field_subset_df_v2[["001", "leader"] + all_keys]

# ####
# # FOR SUBJECTS (650)
# # Parse values into a dictionary-like structure
# field_subset_df_v3["parsed"] = field_subset_df_v3["650"].apply(lambda x: {kv.split('":')[0]: kv.split('":')[1] for kv in x.split("⑄")} if isinstance(x, str) else {})
# # Extract all unique keys (column names)
# all_keys = sorted(set(k for d in field_subset_df_v3["parsed"] for k in d.keys()))
# # all_keys
# # Create new columns with values only (remove key names)
# for key in all_keys:
#     field_subset_df_v3[key] = field_subset_df_v3["parsed"].apply(lambda d: d[key] if key in d else "null")
# # df
# # Keep only relevant columns
# field_subset_df_v4 = field_subset_df_v3[["001", "leader"] + all_keys]


####
# FOR CLASSIFICATION (690)
# Parse values into a dictionary-like structure
field_subset_df_v3["parsed"] = field_subset_df_v3["690"].apply(lambda x: {kv.split('":')[0]: kv.split('":')[1] for kv in x.split("⑄")} if isinstance(x, str) else {})
# Extract all unique keys (column names)
all_keys = sorted(set(k for d in field_subset_df_v3["parsed"] for k in d.keys()))
# all_keys
# Create new columns with values only (remove key names)
for key in all_keys:
    field_subset_df_v3[key] = field_subset_df_v3["parsed"].apply(lambda d: d[key] if key in d else "null")
# df
# Keep only relevant columns
field_subset_df_v4 = field_subset_df_v3[["001", "leader"] + all_keys]

In [ ]:
field_subset_df_v4.info()

In [ ]:
field_subset_df_v4.head(5)

In [ ]:
field_subset_df_v4.shape

In [ ]:
field_subset_df_v5 = field_subset_df_v4.reset_index(drop=True)

In [ ]:
# RENAME COLUMNS
df_columns = field_subset_df_v5.columns
# df_columns
field_subset_df_v5.columns = field_subset_df_v5.columns.str.replace('"', '', regex=False).str.strip()
# field_subset_df_v5.rename(columns={'650': '650', 'leader': 'leader', '0': 'indicator_0', '1': 'indicator_1', '2': 'indicator_2', '4': 'indicator_4', '6': 'indicator_6', '8': 'indicator_8', 'a': 'subfield_a', 'd': 'subfield_d', 'g': 'subfield_g', 'l': 'subfield_l', 'v': 'subfield_v', 'x': 'subfield_x', 'y': 'subfield_y', 'z': 'subfield_z'}, inplace=True) --> FOR SUBJECTS
field_subset_df_v5.rename(columns={'690': '690', 'leader': 'leader', '"a': 'subfield_a', '"b': 'subfield_b', '"c': 'subfield_c'}, inplace=True)
field_subset_df_v5.info()

## Correct problems (if there are)

In [ ]:
# # the lines below check what the problem is when an "empty" column appears, i.e., a column that is not expected, which means something went wrong with the parsing.
# field_subset_df_problem1 = field_subset_df_v5.reset_index(drop=True)

<!-- If the split shows some strange columns, you may need to inspect if it went well, for that it may be useful to check the unique values in the suspicious column, but since column names contain quotation marks, it is best to rename them first: -->

In [ ]:
# # RENAME COLUMNS
# df_columns = field_subset_df_problem1.columns
# # df_columns
# field_subset_df_problem1.columns = field_subset_df_v5.columns.str.replace('"', '', regex=False).str.strip()
# field_subset_df_problem1.rename(columns={'650': '650', 'leader': 'leader', '': 'empty', '0': 'indicator_0', '1': 'indicator_1', '2': 'indicator_2', '4': 'indicator_4', '6': 'indicator_6', '8': 'indicator_8', 'a': 'subfield_a', 'd': 'subfield_d', 'g': 'subfield_g', 'l': 'subfield_l', 'v': 'subfield_v', 'x': 'subfield_x', 'y': 'subfield_y', 'z': 'subfield_z'}, inplace=True)
# field_subset_df_problem1.info()

<!-- Now you can inspect the column's unique values -->

In [ ]:
# field_subset_df_problem1['empty'].unique().tolist()

### Report problematic records (now drop)

<!-- To see which records are problematic based on the previous inspection use: -->

In [ ]:
## DELETE
# to_correct_df = field_subset_df_v5
# to_correct_df.describe()

# to_correct_df[to_correct_df['" "'].notnull()]
# # to_correct_df[to_correct_df['"1"'].notnull()]
# # to_correct_df['"1"'].unique()
# test_record2 = to_correct_df[to_correct_df['"1"'].str.contains("http", case=False, regex=True)]
# test_record2
# to_correct_df2 = to_correct_df.drop(199050).reset_index(drop=True).copy()
# to_correct_df2.info()
# # delete
# to_correct_df3 = to_correct_df2.reset_index(drop=True)


In [ ]:
# # TODO: check what it's in the column with no value " " --> Renamed to "empty"
# # check = field_subset_df_v3[['001','empty']]
# # check_problem.describe()

# check_problem = to_correct_df[~to_correct_df['empty'].str.contains("null", case=False, regex=True)]
# check_problem.head(10)

In [ ]:
# # inspect the problematic record in the originally imported csv
# check_problem_record = biblio_df_v0[biblio_df_v0['001'] == '1529613'] #'1466360'
# check_problem_record

In [ ]:
# # get row number of problematic record in the sliced dataframe
# check_tcn = '1529613'
# check_record = field_subset_df_v5[field_subset_df_v5['001'] == check_tcn]
# check_record

In [ ]:
# REPORT THESE RECORDS TO MIEKE (APRIL 17, 2025)
1529613

In [ ]:
# # TEMPORARILY DROP THE OUTLIER rows which contain the same TCN
# field_subset_df_v6 = field_subset_df_v5.drop(field_subset_df_v5[field_subset_df_v5['001'] == '1529613'].index)
# field_subset_df_v6.info(verbose=True)

In [ ]:
# check_problem_again = field_subset_df_v6[~field_subset_df_v6['empty'].str.contains("null", case=False, regex=True)]
# check_problem_again.head(10)

In [ ]:
# # if it's all corrected, remove the columns that are empty
# field_subset_df_v6.drop(['empty'], axis='columns', inplace=True)
# field_subset_df_v7 = field_subset_df_v6.reset_index(drop=True).copy()
# field_subset_df_v7.info()

## Concatenate the dataframes again (empty/non-empty)

In [ ]:
# CAREFUL HERE: if some problems were corrected, make a copy of version 7, if not, make a copy of version 5 (before correcting: step 4.4)
field_subset_df_v8 = field_subset_df_v5.reset_index(drop=True)

In [ ]:
# special for when there were two separated dfs one with non-empty values one with empty values, this is useful to generate
# report of empty fields, especially for the pie chart with records that don't have any value in 650a

# create column 650 with 'null/notnull' values to be able to filter
field_subset_df_v8["690"] = 'notnull'
frames = [field_subset_df_v8, field_subset_df_vb]
field_subset_df_v9 = pd.concat(frames, ignore_index=True, sort=False)

In [ ]:
# check if it went well
field_subset_df_v9['690'].unique().tolist()

In [ ]:
field_subset_df_v9.info()

In [ ]:
# convert datatypes and fill in empty values
df_columns = field_subset_df_v9.columns
for column in df_columns:
    dataType = field_subset_df_v9.dtypes[column]
    if dataType == np.float64:
        field_subset_df_v9[column] = field_subset_df_v9[column].fillna('null')
        field_subset_df_v9[column] = field_subset_df_v9[column].astype(str)
    if dataType == np.int_:
        biblio_field_subset_df_v9df_v0[column] = field_subset_df_v9[column].fillna('null')
        field_subset_df_v9[column] = field_subset_df_v9[column].astype(str)
    if dataType == object:
        field_subset_df_v9[column] = field_subset_df_v9[column].fillna('null')
        field_subset_df_v9[column] = field_subset_df_v9[column].astype(str)

In [ ]:
field_subset_df_v10 = field_subset_df_v9.reset_index(drop=True).copy()

## Split the leader code
to be able to filter per media type

In [ ]:
# split the leader code to be able to generate report per item type
# split using character position, remember the leader is at position 6 to 8
field_subset_df_v10["leader_code"] = field_subset_df_v10["leader"].map(lambda x: x[5:8])

In [ ]:
# check if it went well
field_subset_df_v10['leader_code'].value_counts()

In [ ]:
# check if it went well
field_subset_df_v10.describe()

## Download slice

In [ ]:
field_subset_df = field_subset_df_v10.reset_index(drop=True)
field_subset_df.info(verbose=True)

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
# name_file = 'biblio_author_person_field_100a' #--> authors test
# name_file = 'biblio_geo_651a' #--> geoterms
# name_file = 'biblio_serials_report_simon' #--> serials report for Simon 2024-11-14
# name_file = 'subjects_600_subfields' # for thesaurus report (202504? and 20250414)
# name_file = 'persons_100_subfields' # for thesaurus report (20250414)
name_file = 'subjects_690_subfields' # for classification report (20251125)

timestr = time.strftime("%Y%m%d-%H%M%S")

# field_subset_df.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
field_subset_df.to_csv(f'{data_downloads_biblio}/{name_file}_{timestr}.csv', index=False) # if too big, use compression='gzip'

In [ ]:
# # You may want to dowload the table above to an excel file for further inspection:

# # choose any name for your file, the file will go to the ../data/downloads folder.
# # name_file = 'biblio_author_person_field_100a' #--> authors test
# # name_file = 'biblio_geo_651a' #--> geoterms
# # name_file = 'authorities_geo_151a_parenthesis'
# # name_file = 'subject_terms_per_150'
# name_file = 'person_names_per_100'

# # field_subset_df.to_excel(f'{data_downloads}/{name_file}.xlsx')

# ## or download to csv
# field_subset_df_v3.to_csv(f'{data_downloads_authority}/{name_file}.csv', index=False) # if too big, use compression='gzip'

# Create a subset of records with a certain value in a given column (toDo)
You may also want to create a list of the records with a certain value in a given column, for example, for field 100e you got these unique values: ['creator.', 'null', 'creator']. You may want to get only the list of records that have "creator."

In [ ]:
# when the file above is too big, it's useful sometimes to download it and upload it here again
path = '/Users/lilianam/workspace/iisg-metadata-overviews/biblio/data/downloads'
# field_subset_df = pd.read_csv(f'{path}/biblio_titles.csv.gz', sep=",", compression='gzip', low_memory=False)

field_subset_df = pd.read_csv(f'{path}/biblio_serials_report_simon.csv', sep=",", low_memory=False)

In [ ]:
field_subset_df.head(5)

In [ ]:
# # check if a string value exists in a column (the string is exactly the same)
# query_value_exact = field_subset_df[field_subset_df['100a'] == 'Hajnal, Henri.']
# query_value_exact

In [ ]:
# check if a string value exists in a column (the string is approximately the same)
# you may want to find the records that have either "creator." (with dot) or "creator" without dot, but not the null values
# here it's possible to use regular expressions

# query_value_aprox = field_subset_df[field_subset_df['852j'].str.contains("ZDF|ZF|ZDK|ZO|XZK|ZDO|ZK", case=True, regex=True)] # for LA periodicals

# query_value_aprox2 = (field_subset_df[field_subset_df['leader'].str.contains('cas|nas') & field_subset_df['852c'].str.contains('NIBG')]) --> Simon report, not good

query_value_aprox3 = field_subset_df[field_subset_df['leader'].str.contains("cas|nas", case=True, regex=True)] # for simon report v3


In [ ]:
query_value_aprox3.head(5)

In [ ]:
# get some idea of how many rows are in this set
query_value_aprox3.info(verbose = True, show_counts = True)

In [ ]:
serials_subset = query_value_aprox3.copy()

In [ ]:
# convert datatypes and fill in empty values
df_columns = serials_subset.columns
for column in df_columns:
    dataType = serials_subset.dtypes[column]
    if dataType == np.float64:
        serials_subset[column] = serials_subset[column].fillna('null')
        serials_subset[column] = serials_subset[column].astype(str)
    if dataType == np.int_:
        serials_subset[column] = serials_subset[column].fillna('null')
        serials_subset[column] = serials_subset[column].astype(str)
    if dataType == object:
        serials_subset[column] = serials_subset[column].fillna('null')
        serials_subset[column] = serials_subset[column].astype(str)

In [ ]:
# check again the number of unique values in your subset
serials_subset.describe()

In [ ]:
serials_subset.info(verbose = True, show_counts = True)

In [ ]:
serials_subset.head(100)

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
# name_file = 'biblio_author_person_field_100a_henri'
# name_file = 'biblio_to_map_la_periodicals_852j'

name_file = 'biblio_serials_simon_report_v4'

# serials_subset.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# query_value_aprox.to_csv()
serials_subset.to_csv(f'{data_downloads}/{name_file}')

# Create subsets using inverse query (toDo)
You may need to create a report with all the records that do not contain a certain value. For example, because we used "null" to fill in all empty values, one could create a list with all the records that have a value in a certain column.

In [ ]:
# create a slice with the records that have non-null values in the column of interest
# Note: if you want to query the subset instead of the whole data, then replace "biblio_df" with "field_subset_df" and run the cell again

query_inverse = biblio_df[~biblio_df['100a'].str.contains("null", case=False, regex=True)]

query_inverse.head(10)

In [ ]:
# get some info about the subset you got as a result of the query:
query_inverse.info(verbose=True, show_counts = True)

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
name_file = 'biblio_author_person_field_100a_notEmpty'

query_inverse.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# query_inverse.to_csv()

# Query for specific records (toDo)
You may want to see the details of specific records, this can be done in two ways:

In [ ]:
# 1. by using the index position. Example: This item: ToDo has index position 0. 
# This position can be seen in the left corner of the entire table (cell above in Section5: biblio_df.head(10))
# We will query it using the entire version of the data, not the subset

# show record vertically using index position
query_recordIndex = biblio_df.iloc[0]
query_recordIndex

In [ ]:
# 2. By using the record Id using the Marc field 001
query_recordId = biblio_df[biblio_df['001'] == '8']
query_recordId

# TEST SQL CONNECTION (toDo)

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

field_subset_df.to_sql(name='testdb', con=engine)

In [ ]:
from sqlalchemy import text
with engine.connect() as conn:
    conn.execute(text("SELECT * FROM testdb")).fetchall()
    [(0, '001'), (1, '245a'), (2, '245b')]


In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("PRAGMA table_info(testdb)")).fetchall()
    print(result)

In [ ]:
## TEST SQL ENDS HERE